In [2]:
from scipy.interpolate import interp1d
from scipy.integrate import simpson
import numpy as np

# 设计船与母船尺度数据

In [3]:
L_total_parent = 225 # 母船总长度
Lpp_parent = 218 # 母船垂线间长
B_parent = 32.26 # 母船型宽
D_parent = 19.6 # 母船型深
d_design_parent = 12.5 # 母船设计吃水
d_structure_parent = 14.3 # 母船结构吃水\
DW_parent = 75088 # 母船载重量 :t
Lw_parent = Lpp_parent+5*0.75 # 水线间长:m ,首垂线与尾垂线距离+ 尾垂线后5个肋位，每个肋位距离为0.75m，参考母船总布置图
L_ball_parent = 4*0.75 # 首垂线至球首前端距，#256 - # 260，肋距为0.75m，参考母船总布置图
nabla_parent = 85488.56 # 母船结构吃水14.3m处的排水体积，参考静水力计算书
delta_parent = 87809.16 # 母船结构吃水处的排水量,参考静水力计算书吃水14.3m时的displt

In [4]:
# 设计船信息
DW = 72100 # 设计船载重量,t
Cb = 0.85
Lpp = 215.17    # m
B = 31.84       # m 
D = 19.34       # m
d = 14.11       # m
delta = 84292   # t

# 截面面积

In [5]:
# 横剖面面积计算
AS_tail = 52.82 * 2 # 艉封板
AS_0 =  (72.45 + 0.82) * 2 # 0站,0.82是螺旋桨位置剖面面积
AS_0_5 = 103.59 * 2 # 0.5站
AS_0_75 = 129.23 * 2 # 0.75站
AS_1 = 1512.21 * 2 # 1站
AS_1_5 = 191.21 * 2 # 1.5站
AS_2 = 225.26 * 2 # 2站
AS_2_5 =  246.50 * 2 # 2.5站
AS_3 = 266.03 * 2 # 3站
AS_3_5 = 282.17 * 2 # 3.5站
AS_4 = 294.87 * 2 # 4站
AS_4_5 = 303.41 * 2 # 4.5站
AS_5 = 308.59 * 2 # 5站
AS_5_5 = 311.41 * 2 # 5.5站
AS_6 = 313.26 * 2 # 6站
AS_6_5 = 314.19 # 6.5站
AS_7 = 314.48 * 2 # 7站
AS_8 = AS_9 = AS_10 = AS_11 = AS_12 = AS_13 = AS_14 = 314.56 * 2
AS_15 = 311.30 * 2 # 15站
AS_16 =  314.38 * 2 # 16站
AS_17 = 312.91 * 2 # 17站截面的面积
AS_17_5 = 302.41 * 2 # 17.5站
AS_18 =  276.11 * 2 # 18站
AS_18_5 = 242.77 * 2 # 18.5站
AS_19 = 199 * 2 # 19站
AS_19_5  = 153.46 * 2 # 19.5站
AS_20 = (19.37 + 31.66) * 2 # 20站

# 货舱

## 母船货舱数据

In [6]:
# 容积
V_cargo1_parent = 10428.53 # m3
V_cargo2_parent = 13612.98
V_cargo3_parent = 13477.53
V_cargo4_parent = 13482.34
V_cargo5_parent = 13571.75
V_cargo6_parent = 13566.95
V_cargo7_parent = 12117.57
# 重心在货舱中的齐次坐标: (重心坐标 - 货舱起始坐标)/货舱长度 (重心坐标 - 货舱起始高度)/货舱高度 横向坐标统一为0
LCG_ratio_cargo1_parent = (194.613 - 183.98)/23.49
LCG_ratio_cargo2_parent = (170.559 - 157.88)/25.23
LCG_ratio_cargo3_parent = (144.513 - 131.78)/25.23
LCG_ratio_cargo4_parent = (118.418 - 105.68)/25.23
LCG_ratio_cargo5_parent = (92.236 - 79.58)/25.23
LCG_ratio_cargo6_parent = (66.130 - 53.48)/25.23
LCG_ratio_cargo7_parent = (40.993 - 28.25)/25.36

VCG_ratio_cargo1_parent = (11.481 - 1.71)/17.89
VCG_ratio_cargo2_parent = (11.084 - 1.71)/17.89
VCG_ratio_cargo3_parent = (11.046 - 1.71)/17.89
VCG_ratio_cargo4_parent = (11.047 - 1.71)/17.89
VCG_ratio_cargo5_parent = (11.052 - 1.71)/17.89
VCG_ratio_cargo6_parent = (11.051 - 1.71)/17.89
VCG_ratio_cargo7_parent = (11.489 - 1.71)/17.89

## 舱容估算

In [7]:
# 母船缩放
k = D * B * Lpp / (D_parent*B_parent*Lpp_parent)
V_cargo1 = V_cargo1_parent * k
V_cargo2 = V_cargo2_parent * k
V_cargo3 = V_cargo3_parent * k
V_cargo4 = V_cargo4_parent * k
V_cargo5 = V_cargo5_parent * k
V_cargo6 = V_cargo6_parent * k
V_cargo7 = V_cargo7_parent * k
print("""
第一货舱的容积为  %.2f m3
第二货舱的容积为  %.2f m3
第三货舱的容积为  %.2f m3
第四货舱的容积为  %.2f m3
第五货舱的容积为  %.2f m3
第六货舱的容积为  %.2f m3
第七货舱的容积为  %.2f m3
"""%(V_cargo1, V_cargo2, V_cargo3, V_cargo4, V_cargo5, V_cargo6, V_cargo7))


第一货舱的容积为  10024.38 m3
第二货舱的容积为  13085.42 m3
第三货舱的容积为  12955.22 m3
第四货舱的容积为  12959.84 m3
第五货舱的容积为  13045.78 m3
第六货舱的容积为  13041.17 m3
第七货舱的容积为  11647.96 m3



## 重心估算

In [8]:
L_cargo1 = L_cargo2 = L_cargo3 = L_cargo4 = L_cargo5 = L_cargo6 = 24.65
L_cargo7 = 23.8
start_cargo1 = 179.6
start_cargo2 = 154.1
start_cargo3 = 128.6
start_cargo4 = 103.1
start_cargo5 = 77.6
start_cargo6 = 52.1
start_cargo7 = 27.45
start_v_cargo = 1.684 # 双层底高度
Height_cargo = 17.657
LCG_cargo1 = start_cargo1 + L_cargo1 * LCG_ratio_cargo1_parent
LCG_cargo2 = start_cargo2 + L_cargo2 * LCG_ratio_cargo2_parent
LCG_cargo3 = start_cargo3 + L_cargo3 * LCG_ratio_cargo3_parent
LCG_cargo4 = start_cargo4 + L_cargo4 * LCG_ratio_cargo4_parent
LCG_cargo5 = start_cargo5 + L_cargo5 * LCG_ratio_cargo5_parent
LCG_cargo6 = start_cargo6 + L_cargo6 * LCG_ratio_cargo6_parent
LCG_cargo7 = start_cargo7 + L_cargo7 * LCG_ratio_cargo7_parent

VCG_cargo1 = start_v_cargo + Height_cargo * VCG_ratio_cargo1_parent
VCG_cargo2 = start_v_cargo + Height_cargo * VCG_ratio_cargo2_parent
VCG_cargo3 = start_v_cargo + Height_cargo * VCG_ratio_cargo3_parent
VCG_cargo4 = start_v_cargo + Height_cargo * VCG_ratio_cargo4_parent
VCG_cargo5 = start_v_cargo + Height_cargo * VCG_ratio_cargo5_parent
VCG_cargo6 = start_v_cargo + Height_cargo * VCG_ratio_cargo6_parent
VCG_cargo7 = start_v_cargo + Height_cargo * VCG_ratio_cargo7_parent
print("""
第一货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
第二货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
第三货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
第四货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
第五货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
第六货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
第七货舱的形心纵向垂向坐标为  (%.2f, %.2f) m3
"""%(LCG_cargo1,VCG_cargo1,LCG_cargo2,VCG_cargo2,
    LCG_cargo3,VCG_cargo3,LCG_cargo4,VCG_cargo4,
    LCG_cargo5,VCG_cargo5,LCG_cargo6,VCG_cargo6,
    LCG_cargo7,VCG_cargo7))


第一货舱的形心纵向垂向坐标为  (190.76, 11.33) m3
第二货舱的形心纵向垂向坐标为  (166.49, 10.94) m3
第三货舱的形心纵向垂向坐标为  (141.04, 10.90) m3
第四货舱的形心纵向垂向坐标为  (115.55, 10.90) m3
第五货舱的形心纵向垂向坐标为  (89.97, 10.90) m3
第六货舱的形心纵向垂向坐标为  (64.46, 10.90) m3
第七货舱的形心纵向垂向坐标为  (39.41, 11.34) m3



In [9]:
# 货物重量
rho = 1/1.26 # 参考书p33也货物的积载系数，V=W * u，u是积载因数，因此rho=W/V=1/u
print("货物密度为%.3f t/m^3"%rho)

货物密度为0.794 t/m^3


# 压载水舱

## 侧压载舱

### 舱容计算

In [10]:
W,H,L = 1.283,10.758,25.5 # 平行中体部分侧压载舱S的尺寸
A1 = 30.678 # 第一压载舱S俯视图面积，从CAD双层底图看
H1 = 10.758 # 第一压载舱S高度
A2 = 29.826 # 第七压载舱S俯视图面积
H2 = 5.764 # 第七压载舱S的上半部分高度
ratio = 48.386 / 137.185 # 第七压载舱S下半与上半侧面积之比，这里认为下半与上半压载舱体积之比等于其侧面积之比
# 体积计算
V_SBWT2 = V_SBWT3 = V_SBWT4 = V_SBWT5 = V_SBWT6 = W * H * L # 单侧压载舱的体积
V_SBWT1 = A1 * H1 # 单侧
V_SBWT7 = A2 * H2 * (1 + ratio)
print("""
第一侧压载舱S,P的容积为  %.2f m3
第二侧压载舱S,P的容积为  %.2f m3
第三侧压载舱S,P的容积为  %.2f m3
第四侧压载舱S,P的容积为  %.2f m3
第五侧压载舱S,P的容积为  %.2f m3
第六侧压载舱S,P的容积为  %.2f m3
第七侧压载舱S,P的容积为  %.2f m3
"""%(V_SBWT1,V_SBWT2,V_SBWT3,V_SBWT4,
    V_SBWT5,V_SBWT6,V_SBWT7))


第一侧压载舱S,P的容积为  330.03 m3
第二侧压载舱S,P的容积为  351.96 m3
第三侧压载舱S,P的容积为  351.96 m3
第四侧压载舱S,P的容积为  351.96 m3
第五侧压载舱S,P的容积为  351.96 m3
第六侧压载舱S,P的容积为  351.96 m3
第七侧压载舱S,P的容积为  232.55 m3



### 重心计算

In [11]:
# 平行中体部分侧压载舱重心为
VCG_SBWT2 = VCG_SBWT3 = VCG_SBWT4 = VCG_SBWT5 = VCG_SBWT6 = VCG_SBWT7 = 10.12 # CAD量取
VCG_SBWT1 = 11.46
# 以下数据从CAD中量取
LCG_SBWT1 = 40.64
LCG_SBWT2 = 64
LCG_SBWT3 = 89.5
LCG_SBWT4 = 115
LCG_SBWT5 = 140.5
LCG_SBWT6 = 166
LCG_SBWT7 = 191.5

print("""
第一侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
第二侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
第三侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
第四侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
第五侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
第六侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
第七侧压载舱S的纵向、垂向坐标为  (%.2f, %.2f) m3
"""%(LCG_SBWT1,VCG_SBWT1,
    LCG_SBWT2,VCG_SBWT2,
    LCG_SBWT3,VCG_SBWT3,
    LCG_SBWT4,VCG_SBWT4,
    LCG_SBWT5,VCG_SBWT5,
    LCG_SBWT6,VCG_SBWT6,
    LCG_SBWT7,VCG_SBWT7))


第一侧压载舱S的纵向、垂向坐标为  (40.64, 11.46) m3
第二侧压载舱S的纵向、垂向坐标为  (64.00, 10.12) m3
第三侧压载舱S的纵向、垂向坐标为  (89.50, 10.12) m3
第四侧压载舱S的纵向、垂向坐标为  (115.00, 10.12) m3
第五侧压载舱S的纵向、垂向坐标为  (140.50, 10.12) m3
第六侧压载舱S的纵向、垂向坐标为  (166.00, 10.12) m3
第七侧压载舱S的纵向、垂向坐标为  (191.50, 10.12) m3



## 顶压载舱

In [12]:
# 容积计算
# 假设顶边舱各截面处的面积比例等于各截面处高度的比例
# 重心相对于顶边舱底部的高度比例等于各截面处高度的比例
BASE_H = 3849 # 中横剖面处的顶边舱高度
BASE_A = 19.88 # 中横剖面处的顶边舱面积
BASE_zg = 18.05 # 中横剖面处的重心高度
BASE_L_Z = 15.496 # 顶边舱底部的高度

# 在第一顶边舱中纵剖面均匀取11个位置测量高度
_x = [178.75,181.3,183.85,186.4,188.95,191.5,194.05,196.6,199.15,201.7,204.25]
_h = [4441,4508,4574,4642,4713,4791,4876,4972,5080,5202,5339]
# 各剖面面积
A = [BASE_A * (i/BASE_H) for i in _h]
zg = [BASE_L_Z + (BASE_zg - BASE_L_Z) * (i / BASE_H) for i in _h]
# 插值
A_inter = interp1d(_x,A)
zg_inter = interp1d(_x,zg)
x_list = np.linspace(178.75,204.25,100)
# 求取体积
A_list = A_inter(x_list)
V_TBWT_1 = simpson(A_list,dx=(204.25-178.75)/100)
# 求取重心垂向坐标
zg_list = zg_inter(x_list)
xg_list = x_list
zg_TBWT_1 = np.sum(zg_list*A_list)/np.sum(A_list)
xg_TBWT_1 = np.sum(xg_list*A_list)/np.sum(A_list)
print("""
第一顶压载舱P,S的舱容为                  %.2f   m3
第一顶压载舱P,S重心的纵向、垂向坐标为    (%.2f, %.2f) m3
"""%(V_TBWT_1,xg_TBWT_1,zg_TBWT_1))


第一顶压载舱P,S的舱容为                  629.11   m3
第一顶压载舱P,S重心的纵向、垂向坐标为    (191.89, 18.71) m3



In [13]:
# 第二第三顶压载舱剖面形状在纵向基本一致,与中纵剖面一致
V_TBWT_2 = V_TBWT_3 = BASE_A * 51 # 25.5是舱室的跨度
LCG_TBWT_2 = 153.88
VCG_TBWT_2 = 17.50
LCG_TBWT_3 = 102.25
VCG_TBWT_3 = 17.42
print("""
第二顶压载舱P,S的舱容为                      %.2f m3
第二顶压载舱P,S的重心纵向、垂向坐标为       (%.2f, %.2f) m3
第三顶压载舱P,S的舱容为                      %.2f m3
第三顶压载舱P,S的重心纵向、垂向坐标为       (%.2f, %.2f) m3
"""%(V_TBWT_2,LCG_TBWT_2,VCG_TBWT_2,
     V_TBWT_3,LCG_TBWT_3,VCG_TBWT_3))


第二顶压载舱P,S的舱容为                      1013.88 m3
第二顶压载舱P,S的重心纵向、垂向坐标为       (153.88, 17.50) m3
第三顶压载舱P,S的舱容为                      1013.88 m3
第三顶压载舱P,S的重心纵向、垂向坐标为       (102.25, 17.42) m3



## 双层底压载舱

In [14]:
from scipy.interpolate import interp1d
from scipy.integrate import simpson
import numpy as np
# 第一双层底压载舱
x = [0,0.5,1,2]
A = [107.142,164.021,188.534,222.39]
xg = [42.29,41.53,41.31,41.03]

A_inter = interp1d(x,A)
xg_inter = interp1d(x,xg)
x_list = np.linspace(0,1.688,100)
# 体积计算
A_list = A_inter(x_list)
V_DBWT_1 = simpson(A_list,dx=1.688/100)
# 形心计算
xg_list = xg_inter(x_list)
zg_list = x_list
LCG_DBWT_1 = np.sum(A_list * xg_list) / xg_list.sum()
VCG_DBWT_1 = np.sum(A_list * zg_list) / zg_list.sum()
print("""
第一压载水舱（左、右）的容积为                %.2f m3
第一压载水舱（左、右）形心的纵向、垂坐标为   (%.2f, %.2f) m3
"""%(V_DBWT_1,LCG_DBWT_1,VCG_DBWT_1))


第一压载水舱（左、右）的容积为                290.72 m3
第一压载水舱（左、右）形心的纵向、垂坐标为   (173.59, 190.28) m3



# 淡水

## 淡水舱

In [15]:
x = [13,14,15,16,17,18,19,19.34]
A = [27.11,32.55,37.04,41.05,44.40,46.90,48.82,49.39]
xg = [4.118,4.020,3.952,3.929,3.894,3.881,3.865,3.861]

A_inter = interp1d(x,A)
xg_inter = interp1d(x,xg)
x_list = np.linspace(13.552,19.34,200)
# 体积计算
A_list = A_inter(x_list)
V = simpson(A_list,dx=(19.34-13.552)/200)
# 重心坐标计算
zg_list = x_list
xg_list = xg_inter(x_list)
zg = np.sum(A_list * zg_list) / np.sum(A_list)
xg = np.sum(A_list * xg_list) / np.sum(A_list)
print("""
淡水舱的容积为             %.2f m3
重心的纵向、垂向坐标为    (%.2f,%.2f) m3
"""%(V,xg,zg))


淡水舱的容积为             239.60 m3
重心的纵向、垂向坐标为    (3.92,16.67) m3



In [16]:
# 到港时
# 高度   容积 
# 14.8   41.31
# 14.2   20.53
# 14.3   23.88  # 基本可认为是到港时的液面高度
x_list=  np.linspace(13.552,14.3,200)
A_list = A_inter(x_list)
V = simpson(A_list, dx=(14.3-13.552)/200)
V

23.88351754143688

In [17]:
# 惯性矩计算
# 高度  惯性矩
# 14    1435.21
# 15    1814.39
# 重心坐标计算
zg_list = x_list
xg_list = xg_inter(x_list)
zg = np.sum(A_list * zg_list) / np.sum(A_list)
xg = np.sum(A_list * xg_list) / np.sum(A_list)
I = 1435.21 + (1814.39 - 1435.21) * (14.8 - 14)
print("""
到港时，淡水舱
重心的纵向、垂向坐标为   (%.2f, %.2f) m3
此时的液面最高点高度为     14.8 m
自由液面惯性矩为          %.2f t*m
"""%(xg,zg,I))


到港时，淡水舱
重心的纵向、垂向坐标为   (4.03, 13.93) m3
此时的液面最高点高度为     14.8 m
自由液面惯性矩为          1738.55 t*m



## 蒸馏水舱

In [18]:
V_DIS_WTK = 1.493 * 7.896 * 1.974 # x方向跨度为1.493m，y方向跨度为7.896m，z方向跨度为1.974m,对称于x轴
I_DIS_WTK = 1.493 * 7.896**3 / 12 * 1 # 1是密度,1t/m3
LCG_DIS_WTK = 7.51 # m
VCG_DIS_WTK = 14.51 # m
print("""
蒸馏水舱的容积为        %.2f m3
重心纵向、垂向坐标为   (%.2f, %.2f) m3
自由液面惯性矩为        %.2f t*m
"""%(V_DIS_WTK,LCG_DIS_WTK,VCG_DIS_WTK,I_DIS_WTK))


蒸馏水舱的容积为        23.27 m3
重心纵向、垂向坐标为   (7.51, 14.51) m3
自由液面惯性矩为        61.25 t*m



In [19]:
# 到港时
VCG_DIS_WTK_1_10 = 13.522 + 0.05 * 1.974 # 13.522是蒸馏水舱最底部的高度，1.974是蒸馏水舱的高度，重心在5%处
print("""
到港时，蒸馏水舱
重心纵向、垂向坐标为    (%.2f, %.2f) m3
自由液面惯性矩为         %.2f   t*m
"""%(LCG_DIS_WTK,VCG_DIS_WTK_1_10,I_DIS_WTK))


到港时，蒸馏水舱
重心纵向、垂向坐标为    (7.51, 13.62) m3
自由液面惯性矩为         61.25   t*m



# 燃油

## 柴油舱

In [20]:
# 柴油舱设计容量取130m3，大于计算所需的123.39m3
V_doil = 130
# 总布置图中柴油舱分为左右两舱，每舱容积65m3
# 有CAD测量或测质心数据为
x_doil = 25.37
z_doil = 3.86
print(f"""
柴油舱的容积为             {V_doil} m3
质心的纵向、垂向坐标为     ({x_doil},{z_doil}) m3
""")


柴油舱的容积为             130 m3
质心的纵向、垂向坐标为     (25.37,3.86) m3



In [21]:
# 柴油舱的尺度数据
A_chai = 6.315 # 纵剖面面积:m2
B_chai = V_doil / A_chai # m
L_chai = 1.6 # m
print("""
柴油舱长度为    %.2f m
柴油舱宽度为    %.2f m
"""%(L_chai, B_chai))


柴油舱长度为    1.60 m
柴油舱宽度为    20.59 m



In [22]:
# 柴油舱自由液面惯性矩为
rho_chai = 0.85
I_chai = L_chai * B_chai ** 3 /12 * rho_chai 
print("柴油舱的自由液面惯性矩为 %.2f t*m"%I_chai)

柴油舱的自由液面惯性矩为 988.71 t*m


In [23]:
# 到港时
# 液面最高点高度   体积
# 2.695            26.56
# 2.295            13.38 可近似认为是到港时的液面高度
zg = 2.06
xg = 26.02
print("""
到港时，柴油舱
质心的纵向、垂向坐标为     (%.2f, %.2f) m3
自由液面惯性矩为            %.2f  t*m
"""%(xg,zg,I_chai))


到港时，柴油舱
质心的纵向、垂向坐标为     (26.02, 2.06) m3
自由液面惯性矩为            988.71  t*m



## 燃油沉积舱

In [24]:
V_depo_oil1 = V_depo_oil2 = 30 # 设计燃油沉降容积为30m3
A_depo_oil1 = 10.55
A_depo_oil2 = 10.34
# 横向布置方面从总布置测量获得
x_depo_oil1 = 14.76 # 第一燃油沉积舱
x_depo_oil2 = 22.73 # 第二燃油沉积舱
z_depo_oil1 = 9.732 + (V_depo_oil1/A_depo_oil1) * 0.5
z_depo_oil2 = 9.732 + (V_depo_oil2/A_depo_oil2) * 0.5
print("""
燃油沉积的容积为                    %.2f m3
第一燃油沉积舱的纵向、垂向位置为    (%.2f,%.2f) m3
第二燃油沉积舱的纵向、垂向位置为    (%.2f,%.2f) m3
"""%(V_depo_oil1+V_depo_oil2,
    x_depo_oil1,z_depo_oil1,
    x_depo_oil2,z_depo_oil2))


燃油沉积的容积为                    60.00 m3
第一燃油沉积舱的纵向、垂向位置为    (14.76,11.15) m3
第二燃油沉积舱的纵向、垂向位置为    (22.73,11.18) m3



In [25]:
# 到港时
zg_depo_oil1 = 9.732 + (V_depo_oil1*0.1 / A_depo_oil1) * 0.5
zg_depo_oil2 = 9.732 + (V_depo_oil2*0.1 / A_depo_oil2) * 0.5
I1 = 1377 * 0.89 # CAD测量的惯性矩*燃油密度
I2 = 1342 * 0.89
print("""
到港时，燃油沉积舱
舱一重心的纵向、垂向位置为    (%.2f, %.2f) m
舱二重心的纵向、垂向位置为    (%.2f, %.2f) m
舱一的自由液面惯性矩为         %.2f  t*m
舱二的自由液面惯性矩为         %.2f  t*m
"""%(x_depo_oil1,zg_depo_oil1,
    x_depo_oil2,zg_depo_oil2,
     I1,I2))


到港时，燃油沉积舱
舱一重心的纵向、垂向位置为    (14.76, 9.87) m
舱一重心的纵向、垂向位置为    (22.73, 9.88) m
舱一的自由液面惯性矩为         1225.53  t*m
舱二的自由液面惯性矩为         1194.38  t*m



## 燃油舱

### 第一燃油舱

In [28]:
# 第一燃油舱，高度9.732-18.9范围内
x = list(range(9,20))
A = [20.88, 31.40, 41.89, 51.77, 60.14, 67.17, 73.05, 77.85, 81.49,83.84,84.93]
xg = [22.966,22.045,21.24,20.60,20.21,19.94,19.73,19.56,19.43,19.34,19.30]

A_inter = interp1d(x,A)
xg_inter = interp1d(x,xg)
x_list = np.linspace(9.732,18.9,100)
# 体积计算
A_list = A_inter(x_list)
V_oil1 = simpson(A_list,dx=(18.9-9.732)/100)
# 重心计算
xg_list = xg_inter(x_list)
zg_list = x_list
LCG_oil1 = np.sum(A_list * xg_list) / A_list.sum()
VCG_oil1 = np.sum(A_list * zg_list) / A_list.sum()
print("""
第四燃油舱（左、右）的容积为                   %.2f m3
第四燃油舱（左、右）的纵向、垂向坐标为         (%.2f, %.2f) m3
"""%(V_oil1,LCG_oil1,VCG_oil1))


第四燃油舱（左、右）的容积为                   588.96 m3
第四燃油舱（左、右）的纵向、垂向坐标为         (19.94, 15.00) m3



In [29]:
# 到港时
# h        V
# 11.2    52.73
# 11.3    57.12
# 11.34   58.9
x_list = np.linspace(9.732,11.34,100)
A_list = A_inter(x_list)
V_oil1_1_10 = simpson(A_list,dx=(11.34-9.732)/100)
V_oil1_1_10

58.90111702157576

In [30]:
# h      惯性矩
# 11     5480
# 12     7003
# 惯性矩计算
I = 5480 + (7003 - 5480) * (11.34 - 11)
I *= 0.89
# 重心计算
x_list =np.linspace(9.732,11.34,100)
xg_list = xg_inter(x_list)
zg_list = x_list
LCG_oil1_1_10 = np.sum(A_list * xg_list) / A_list.sum()
VCG_oil1_1_10 = np.sum(A_list * zg_list) / A_list.sum()
print("""
到港时，
重心位置为（纵、垂）  (%.2f, %.2f) m
自由液面惯性矩为       %.2f t*m
"""%(LCG_oil1_1_10,VCG_oil1_1_10,I))


到港时，
重心位置为（纵、垂）  (21.57, 10.60) m
自由液面惯性矩为       5338.06 t*m



### 第二燃油舱

In [31]:
# 第二燃油舱 位于 2 - 8站之间
x = [21.517,26.896,32.2755,43.034,53.7925,64.551,75.3095,86.068]
A = [19.81,21.04,20.04,19.95,19.95,19.95,19.95,19.95]
zg = [18,17.95,18.01,18.02,18.02,18.02,18.02,18.02]

A_inter = interp1d(x,A)
zg_inter = interp1d(x,zg)
x_list = np.linspace(27.45,76.75,100)
# 体积计算
A_list = A_inter(x_list)
V_oil2 = simpson(A_list,dx=(76.75-27.45)/100)
# 重心计算
xg_list = x_list
zg_list = zg_inter(x_list)
LCG_oil2 = np.sum(A_list * xg_list) / A_list.sum()
VCG_oil2 = np.sum(A_list * zg_list) / A_list.sum()
print("""
第三燃油舱（左、右）的容积为               %.2f m3
第三燃油舱（左、右）的纵向、垂向坐标为     (%.2f, %.2f)m3
"""%(V_oil2,LCG_oil2,VCG_oil2))


第三燃油舱（左、右）的容积为               976.75 m3
第三燃油舱（左、右）的纵向、垂向坐标为     (52.03, 18.01)m3



In [32]:
# 到港时
# 近似认为纵向方向各个剖面的面积相同，则体积之比相当于液体剖面积之比
# 满载时的面积19.82
# 高度     面积
# 16.41   1.936
# 16.44   1.97
zg = 16.03 # 液面16.44时的重心高度
I = 14609 * 0.89 # CAD量取
print("""
到港时，第二燃油舱
重心的纵向、垂向坐标为    (%.2f, %.2f) m
自由液面惯性矩为           %.2f  t*m
"""%(LCG_oil2,zg,I))


到港时，第二燃油舱
重心的纵向、垂向坐标为    (52.03, 16.03) m
自由液面惯性矩为           13002.01  t*m



### 第三燃油舱

In [33]:
A = 20 # 俯视图面积
H = 5.5 # 高度
V_oil3 = A * H # 第三燃油舱容积
LCG_oil3 = 22.45 # CAD量取
VCG_oil3 = 9.774 + 0.5 * H
print("""
第三燃油舱的容积为        %.2f m3
中心的纵向、垂向坐标为   (%.2f, %.2f) m3
"""%(V_oil3,LCG_oil3,VCG_oil3))


第三燃油舱的容积为        110.00 m3
中心的纵向、垂向坐标为   (22.45, 12.52) m3



In [34]:
# 到港时
H = 0.55
VCG_oil3 = 9.774 + 0.05 * H
I = 1226 * 0.89
print("""
到港时，第三燃油舱
重心的纵向、垂向坐标为    (%.2f, %.2f) m
自由液面惯性矩为           %.2f t*m
"""%(LCG_oil3,VCG_oil3,I))


到港时，第三燃油舱
重心的纵向、垂向坐标为    (22.45, 9.80) m
自由液面惯性矩为           1091.14 t*m



# 滑油

In [65]:
# 滑油循环舱
L,W,H = 8.686,1.974,0.987
V_HC = L * W * H
LCG_HC = 20.275
VCG_HC = 1.125
# 滑油污油舱
x = [0,0.5,1,2] # 水线高度
xg = [25.394,24.908,24.839,24.77] # 重心x坐标
A = [5.56,12.678,15.66,20.338] # 面积
xg_inter = interp1d(x,xg)
A_inter = interp1d(x,A)
zg_list = np.linspace(0.632,1.619,num=100)
# 体积计算
A_list = A_inter(zg_list)
V_HW = simpson(A_list,dx=(1.619-0.632)/100)
# 重心计算
xg_list = xg_inter(zg_list)
LCG_HW = np.sum(A_list * xg_list) / A_list.sum()
VCG_HW = np.sum(A_list * zg_list) / A_list.sum()
print("""
滑油循环舱的容积为   %.2f m3
滑油循环舱的纵向、垂向坐标为  (%.2f, %.2f) m3
滑油污油舱的容积为   %.2f m3
滑油污油舱的纵向、垂向坐标为  (%.2f, %.2f) m3
"""%(V_HC,LCG_HC,VCG_HC,
     V_HW,LCG_HW,VCG_HW))


滑油循环舱的容积为   16.92 m3
滑油循环舱的纵向、垂向坐标为  (20.27, 1.12) m3
滑油污油舱的容积为   15.79 m3
滑油污油舱的纵向、垂向坐标为  (24.83, 1.15) m3



In [66]:
# 到港时
# 滑油循环舱
zg = 0.632 + 0.987*0.05
I = L * W**3 /12 * 0.9
print("""
到港时，滑油循环舱
重心纵向、垂向坐标为   (%.2f, %.2f)m
自由液面惯性矩为        %.2f t*m
"""%(LCG_HC,zg,I))


到港时，滑油循环舱
重心纵向、垂向坐标为   (20.27, 0.68)m
自由液面惯性矩为        5.01 t*m



In [74]:
# 到港时，滑油污油舱
# h   V
# 0.8  2.323
# 0.74  1.474 # 基本等于到港时的滑油污油容积
# 0.72 1.196
zg_list = np.linspace(0.632,0.75,num=100)
A_list = A_inter(zg_list)
simpson(A_list,dx=(0.75-0.632)/100)

1.61411642568

In [75]:
zg_list = np.linspace(0.632,0.75,num=100)
# 体积计算
A_list = A_inter(zg_list)
V_HW = simpson(A_list,dx=(0.75-0.632)/100)
# 重心计算
xg_list = xg_inter(zg_list)
LCG_HW = np.sum(A_list * xg_list) / A_list.sum()
VCG_HW = np.sum(A_list * zg_list) / A_list.sum()
I = 268 * 0.9
print("""
到港时，滑油污油舱
重心的纵向、横向坐标为      (%.2f, %.2f) m
自由液面惯性矩为             %.2f  t*m
"""%(LCG_HW,VCG_HW,I))


到港时，滑油污油舱
重心的纵向、横向坐标为      (24.88, 0.69) m
自由液面惯性矩为             241.20  t*m



# 污水和其他

## 舱底水分离舱

In [24]:
z_range = [0,1.619]
zg_list = np.linspace(*z_range,num=100)

x = [0,0.5,1,2]
xg = [18.99,18.906,18.891,18.885]
A = [8.088,15.664,18.935,23.461]
xg_inter = interp1d(x,xg)
A_inter = interp1d(x,A)
# 体积计算
A_list = A_inter(zg_list)
V_BWF = simpson(A_list,dx=(z_range[1] - z_range[0])/100)
# 重心计算
xg_list = xg_inter(zg_list)
LCG_BWF = np.sum(A_list * xg_list) / A_list.sum()
VCG_BWF = np.sum(A_list * zg_list) / A_list.sum()
print("""
舱底水分离舱的容积为     %.2f m3
舱底水分离舱重心的横向坐标为  (%.2f, %.2f) m3
"""%(V_BWF,LCG_BWF,VCG_BWF))


舱底水分离舱的容积为     26.90 m3
舱底水分离舱重心的横向坐标为  (18.90, 0.91) m3



## 燃油污油舱

In [25]:
print("""
燃油污油舱的容积为   %.2f m3
燃油污油舱的纵向、垂向坐标为  (%.2f, %.2f) m3
"""%(V_HW,LCG_HW,VCG_HW))


燃油污油舱的容积为   15.79 m3
燃油污油舱的纵向、垂向坐标为  (24.83, 1.15) m3



# 生活用品

In [26]:
# 获取重量，估计坐标
food_weight = 6.76
spares_weight = 93.27
luggage_weight = 1.26
LCG_food, VCG_food = 13.05,23.14
LCG_spares, VCG_spares = 18.194,23.14
LCG_luggage, VCG_luggate = 16.25,26.2

# 空船重量与重心

In [27]:
# 空船重量为
W_empty_ship = 12498.28
# 重心数据利用母船进行缩放
LCG_empty_ship = 101.10 * Lpp / Lpp_parent # 指导书p94
VCG_empty_ship = 11.66 * D / D_parent
print("""
空船重量为               %.2f t
空船的纵向、横向坐标为  (%.2f, %.2f) m
"""%(W_empty_ship, LCG_empty_ship, VCG_empty_ship))


空船重量为               12498.28 t
空船的纵向、横向坐标为  (99.79, 11.51) m

